# BÀI TẬP THỰC HÀNH 1
## Thực hiện thống kê mô tả trên tập dữ liệu về phân loại chất lượng rượu đỏ
Nguồn dữ liệu: [Red Wine Quality Dataset - Kaggle](https://www.kaggle.com/code/eisgandar/red-wine-quality-eda-classification)


# 1. Nhập thư viện

In [52]:
import pandas as pd
import numpy as np
from scipy import stats

# 2. Đọc dữ liệu
Đọc tập dữ liệu winequality-red.csv vào DataFrame wine.

In [53]:
wine = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/EDA/winequality-red.csv", sep=",")

# 3. Thông tin tổng quan



In [54]:
# shape
print(f'+ Shape: {wine.shape}')
# types
print(f'+ Data Types: \n{wine.dtypes}')
# head, tail
print(f'+ Contents: ')
display(wine.head(5))
display(wine.tail(5))
# info
wine.info()
desc = wine.describe().T


+ Shape: (1599, 12)
+ Data Types: 
fixed acidity           float64
volatile acidity        float64
citric acid             float64
residual sugar          float64
chlorides               float64
free sulfur dioxide     float64
total sulfur dioxide    float64
density                 float64
pH                      float64
sulphates               float64
alcohol                 float64
quality                   int64
dtype: object
+ Contents: 


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5
1598,6.0,0.310,0.47,3.6,0.067,18.0,42.0,0.99549,3.39,0.66,11.0,6


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


Các biến có quy mô giá trị khác nhau, ví dụ residual sugar và density dao động khác biệt. alcohol có giá trị trung bình khoảng 10.4%, điều này ảnh hưởng mạnh đến chất lượng rượu.

# 4. Kiểm tra dữ liệu thiếu / giá trị bất hợp lý

In [55]:
missing = wine.isnull().sum()
n_duplicated = wine.duplicated().sum()

missing_percent = (missing / len(wine)) * 100
missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Missing Percent': missing_percent
})
display(missing_df)
print(f'+ Số dòng trùng: {n_duplicated}')

,Missing Count,Missing Percent
fixed acidity,0,0.0
volatile acidity,0,0.0
citric acid,0,0.0
residual sugar,0,0.0
chlorides,0,0.0
free sulfur dioxide,0,0.0
total sulfur dioxide,0,0.0
density,0,0.0
pH,0,0.0
sulphates,0,0.0


+ Số dòng trùng: 240


Tập dữ liệu gồm 1359 dòng và 12 cột — mỗi dòng đại diện cho một mẫu rượu, các cột là các chỉ số hóa học hoặc chất lượng cảm quan. Không có dữ liệu bị thiếu, tuy nhiên có vài cột có giá trị 0 (như citric acid hoặc residual sugar), cần cân nhắc khi phân tích.

# 5. Xử lý dữ liệu thiếu

In [56]:
numeric_cols = wine.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    median_val = wine[col].median()
    wine.loc[wine[col] < 0, col] = median_val

wine = wine.drop_duplicates()
for col in numeric_cols:
    if wine[col].isnull().sum() > 0:
        # thay bằng median
        wine[col].fillna(wine[col].median(), inplace=True)

print("\nKiểm tra dữ liệu thiếu sau khi xử lý:")
print(wine.isnull().sum())
print("\nSau khi xóa trùng lặp:")
print("Số dòng còn lại:", wine.shape[0])


Kiểm tra dữ liệu thiếu sau khi xử lý:
fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

Sau khi xóa trùng lặp:
Số dòng còn lại: 1359


# 6. Bổ sung độ lệch và độ nhọn

In [57]:
from scipy.stats import skew, kurtosis
desc["skewness"] = wine.skew()
desc["kurtosis"] = wine.kurtosis()
print(desc)

                       count       mean        std      min      25%  \
fixed acidity         1599.0   8.319637   1.741096  4.60000   7.1000   
volatile acidity      1599.0   0.527821   0.179060  0.12000   0.3900   
citric acid           1599.0   0.270976   0.194801  0.00000   0.0900   
residual sugar        1599.0   2.538806   1.409928  0.90000   1.9000   
chlorides             1599.0   0.087467   0.047065  0.01200   0.0700   
free sulfur dioxide   1599.0  15.874922  10.460157  1.00000   7.0000   
total sulfur dioxide  1599.0  46.467792  32.895324  6.00000  22.0000   
density               1599.0   0.996747   0.001887  0.99007   0.9956   
pH                    1599.0   3.311113   0.154386  2.74000   3.2100   
sulphates             1599.0   0.658149   0.169507  0.33000   0.5500   
alcohol               1599.0  10.422983   1.065668  8.40000   9.5000   
quality               1599.0   5.636023   0.807569  3.00000   5.0000   

                           50%        75%        max  skewness 

**Nhận xét**:

Bổ sung độ lệch (skewness) và độ nhọn (kurtosis) giúp hiểu sâu hơn về hình dạng phân phối dữ liệu.
Một số biến như residual sugar và chlorides có độ lệch dương lớn → phân phối lệch phải mạnh, chứa nhiều giá trị ngoại lai.
Các biến có kurtosis cao (>3) cần được chuẩn hóa hoặc log-transform để giảm ảnh hưởng của outlier.

# 7. Thống kê mô tả


In [58]:
stats_summary = {}

for col in numeric_cols:
    data = wine[col]

    mean_val = np.mean(data)
    median_val = np.median(data)
    mode_val = stats.mode(data, keepdims=True)[0][0]
    var_val = np.var(data, ddof=1)  # variance
    std_val = np.std(data, ddof=1)  # standard deviation
    range_val = np.max(data) - np.min(data)
    q1 = np.percentile(data, 25)
    q3 = np.percentile(data, 75)
    iqr = q3 - q1

    stats_summary[col] = {
        "Mean": mean_val,
        "Median": median_val,
        "Mode": mode_val,
        "Variance": var_val,
        "Std Dev": std_val,
        "Range": range_val,
        "Q1 (25%)": q1,
        "Q3 (75%)": q3,
        "IQR": iqr
    }

In [59]:
stats_df = pd.DataFrame(stats_summary).T
print("\nThống kê mô tả:")
display(stats_df)


Thống kê mô tả:


,Mean,Median,Mode,Variance,Std Dev,Range,Q1 (25%),Q3 (75%),IQR
fixed acidity,8.310596,7.9000,7.2000,3.017134,1.736990,11.30000,7.1000,9.20000,2.10000
volatile acidity,0.529478,0.5200,0.5000,0.033500,0.183031,1.46000,0.3900,0.64000,0.25000
citric acid,0.272333,0.2600,0.0000,0.038235,0.195537,1.00000,0.0900,0.43000,0.34000
residual sugar,2.523400,2.2000,2.0000,1.828752,1.352314,14.60000,1.9000,2.60000,0.70000
chlorides,0.088124,0.0790,0.0800,0.002438,0.049377,0.59900,0.0700,0.09100,0.02100
free sulfur dioxide,15.893304,14.0000,6.0000,109.145456,10.447270,71.00000,7.0000,21.00000,14.00000
total sulfur dioxide,46.825975,38.0000,28.0000,1116.157653,33.408946,283.00000,22.0000,63.00000,41.00000
density,0.996709,0.9967,0.9968,0.000003,0.001869,0.01362,0.9956,0.99782,0.00222
pH,3.309787,3.3100,3.3000,0.024036,0.155036,1.27000,3.2100,3.40000,0.19000
sulphates,0.658705,0.6200,0.5400,0.029127,0.170667,1.67000,0.5500,0.73000,0.18000


# 8. So sánh trung bình theo nhóm chất lượng

In [60]:
wine["quality_label"] = wine["quality"].apply(lambda q: "good" if q >= 6 else "bad")
print(wine["quality_label"].value_counts())
print(wine["quality_label"].value_counts(normalize=True))

quality_label
good    719
bad     640
Name: count, dtype: int64
quality_label
good    0.529065
bad     0.470935
Name: proportion, dtype: float64


In [61]:
groups = wine.groupby("quality_label")
for name, group in groups:
    print(f"==== {name} ====")
    print(group.describe().T)

==== bad ====
                      count       mean        std      min      25%  \
fixed acidity         640.0   8.141719   1.561787  4.60000   7.1000   
volatile acidity      640.0   0.592930   0.184137  0.18000   0.4600   
citric acid           640.0   0.237547   0.184963  0.00000   0.0800   
residual sugar        640.0   2.526797   1.339649  1.20000   1.9000   
chlorides             640.0   0.094144   0.058115  0.03900   0.0740   
free sulfur dioxide   640.0  16.659375  10.875864  3.00000   8.0000   
total sulfur dioxide  640.0  55.146094  36.996713  6.00000  23.7500   
density               640.0   0.997044   0.001596  0.99256   0.9961   
pH                    640.0   3.309016   0.154503  2.74000   3.2000   
sulphates             640.0   0.620484   0.181235  0.33000   0.5275   
alcohol               640.0   9.920781   0.773478  8.40000   9.4000   
quality               640.0   4.885937   0.364018  3.00000   5.0000   

                            50%       75%        max  
fixed a

**Nhận xét:**

Rượu “High” có alcohol cao hơn, volatile acidity thấp hơn rõ rệt so với nhóm “Low”. Đây là các yếu tố ảnh hưởng chính đến chất lượng cảm quan.

# 9. Thống kê mô tả chi tiết thủ công cho từng cột số

In [62]:
data = wine.select_dtypes(include=[np.number])

for col in data.columns:
    values = data[col].values

    mean = np.mean(values)
    median = np.median(values)
    mode = stats.mode(values, keepdims=True)   # mode trả về object
    var = np.var(values, ddof=1)               # sample variance (ddof=1)
    std = np.std(values, ddof=1)               # sample standard deviation
    data_range = np.max(values) - np.min(values)
    q1 = np.percentile(values, 25)
    q3 = np.percentile(values, 75)
    iqr = q3 - q1

    print(f"\n=== {col} ===")
    print(f"Mean: {mean:.4f}")
    print(f"Median: {median:.4f}")
    print(f"Mode: {mode.mode[0]} (Count={mode.count[0]})")
    print(f"Variance: {var:.4f}")
    print(f"Standard Deviation: {std:.4f}")
    print(f"Range: {data_range:.4f}")
    print(f"Q1 (25%): {q1:.4f}")
    print(f"Q3 (75%): {q3:.4f}")
    print(f"IQR: {iqr:.4f}")


=== fixed acidity ===
Mean: 8.3106
Median: 7.9000
Mode: 7.2 (Count=49)
Variance: 3.0171
Standard Deviation: 1.7370
Range: 11.3000
Q1 (25%): 7.1000
Q3 (75%): 9.2000
IQR: 2.1000

=== volatile acidity ===
Mean: 0.5295
Median: 0.5200
Mode: 0.5 (Count=37)
Variance: 0.0335
Standard Deviation: 0.1830
Range: 1.4600
Q1 (25%): 0.3900
Q3 (75%): 0.6400
IQR: 0.2500

=== citric acid ===
Mean: 0.2723
Median: 0.2600
Mode: 0.0 (Count=118)
Variance: 0.0382
Standard Deviation: 0.1955
Range: 1.0000
Q1 (25%): 0.0900
Q3 (75%): 0.4300
IQR: 0.3400

=== residual sugar ===
Mean: 2.5234
Median: 2.2000
Mode: 2.0 (Count=133)
Variance: 1.8288
Standard Deviation: 1.3523
Range: 14.6000
Q1 (25%): 1.9000
Q3 (75%): 2.6000
IQR: 0.7000

=== chlorides ===
Mean: 0.0881
Median: 0.0790
Mode: 0.08 (Count=50)
Variance: 0.0024
Standard Deviation: 0.0494
Range: 0.5990
Q1 (25%): 0.0700
Q3 (75%): 0.0910
IQR: 0.0210

=== free sulfur dioxide ===
Mean: 15.8933
Median: 14.0000
Mode: 6.0 (Count=121)
Variance: 109.1455
Standard Deviatio

**Nhận xét**:

Các biến có độ lệch cao như residual sugar, chlorides có phân bố không chuẩn.

Các đặc trưng như alcohol có Mean > Median, cho thấy xu hướng lệch phải nhẹ.

IQR nhỏ ở pH → dữ liệu tập trung quanh trung bình, ít biến động.

Có thể chuẩn hóa hoặc log-transform một số biến để giảm ảnh hưởng outlier.